In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz

--2022-10-16 10:17:18--  https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274079476 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.0-bin-hadoop2.tgz.1’

spark-3.3.0-bin-had 100%[===================>] 261.38M   185MB/s    in 1.4s    

2022-10-16 10:17:19 (185 MB/s) - ‘spark-3.3.0-bin-hadoop2.tgz.1’ saved [274079476/274079476]



In [ ]:
!tar xf spark-3.3.0-bin-hadoop2.tgz

In [ ]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.3.0-bin-hadoop2"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Colab").config("spark.ui.port",'4050').config('spark.dirver.maxResultSize', '4096').getOrCreate()

In [ ]:
#Importing the library
from pyspark.sql.types import IntegerType, DoubleType, DateType
from pyspark.sql.functions import isnan, when, count, col, lit, to_date
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.mllib.stat import Statistics
from pyspark.ml.stat import Correlation
import pyspark.pandas as ps
import matplotlib.pyplot as plt


In [ ]:
#Read the csv file
data = spark.read.csv("/content/drive/MyDrive/CSCI316/data.csv", inferSchema=True, header=True)

In [ ]:
data.show(10)

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+----------+----------+--------------------+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+---------

In [ ]:
pandas_df = data.pandas_api()

In [ ]:
type(pandas_df)

pyspark.pandas.frame.DataFrame

In [ ]:
# plt.boxplot(pandas_df, widths=0.2)
# # plt.xticks([1,2],['Our data', 'Hypothetical data'])
# # plt.ylabel('Grade')
# plt.title('Box plot of midterm grade')
# plt.show()

In [ ]:
#get the percentage of null values 
amount_missing_df = data.select([((count(when(col(c).isNull(), c))/data.count())*100).alias(c) for c in data.columns])
amount_missing_df.show()

+---+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+------------------+----------+--------------+----------+-------------------+-------+----------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+------------------+--------------------+-----------------+--------------------+--------------------------+---------------------------+--------------------+--------------------+----------------+-----------------+-------------------------+--------------------+--

In [ ]:
#Show the types
data.dtypes

[('id', 'int'),
 ('member_id', 'int'),
 ('loan_amnt', 'int'),
 ('funded_amnt', 'int'),
 ('funded_amnt_inv', 'double'),
 ('term', 'string'),
 ('int_rate', 'double'),
 ('installment', 'double'),
 ('grade', 'string'),
 ('sub_grade', 'string'),
 ('emp_title', 'string'),
 ('emp_length', 'string'),
 ('home_ownership', 'string'),
 ('annual_inc', 'string'),
 ('verification_status', 'string'),
 ('issue_d', 'string'),
 ('pymnt_plan', 'string'),
 ('desc', 'string'),
 ('purpose', 'string'),
 ('title', 'string'),
 ('zip_code', 'string'),
 ('addr_state', 'string'),
 ('dti', 'string'),
 ('delinq_2yrs', 'string'),
 ('earliest_cr_line', 'string'),
 ('inq_last_6mths', 'string'),
 ('mths_since_last_delinq', 'string'),
 ('mths_since_last_record', 'string'),
 ('open_acc', 'string'),
 ('pub_rec', 'string'),
 ('revol_bal', 'string'),
 ('revol_util', 'string'),
 ('total_acc', 'string'),
 ('initial_list_status', 'string'),
 ('out_prncp', 'string'),
 ('out_prncp_inv', 'string'),
 ('total_pymnt', 'string'),
 ('t

In [ ]:
#delete the row that has null value in default_ind
delete_default_ind_null = data.filter(data.default_ind.isNotNull())
delete_default_ind_null.select([count(when(col(c).isNull(), c)).alias(c) for c in delete_default_ind_null.columns]).show()

+---+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+---------+----------+--------------+----------+-------------------+-------+----------+------+-------+-----+--------+----------+---+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+-----------+
| id|member_id|loan_amnt|funded_amnt|funded_amnt_

In [ ]:
#Deal with outliers
def find_outliers(df):

    # Identifying the numerical columns in a spark dataframe
    numeric_columns = [column[0] for column in df.dtypes if column[1]=='int']

    # Using the `for` loop to create new columns by identifying the outliers for each feature
    for column in numeric_columns:

        less_Q1 = 'less_Q1_{}'.format(column)
        more_Q3 = 'more_Q3_{}'.format(column)
        Q1 = 'Q1_{}'.format(column)
        Q3 = 'Q3_{}'.format(column)

        # Q1 : First Quartile ., Q3 : Third Quartile
        Q1 = df.approxQuantile(column,[0.25],relativeError=0)
        Q3 = df.approxQuantile(column,[0.75],relativeError=0)
        
        # IQR : Inter Quantile Range
        # We need to define the index [0], as Q1 & Q3 are a set of lists., to perform a mathematical operation
        # Q1 & Q3 are defined seperately so as to have a clear indication on First Quantile & 3rd Quantile
        IQR = Q3[0] - Q1[0]
        
        #selecting the data, with -1.5*IQR to + 1.5*IQR., where param = 1.5 default value
        less_Q1 =  Q1[0] - 1.5*IQR
        more_Q3 =  Q3[0] + 1.5*IQR
        
        isOutlierCol = 'is_outlier_{}'.format(column)
        
        df = df.withColumn(isOutlierCol, when((df[column] > more_Q3) | (df[column] < less_Q1), 1).otherwise(0))
    

    # Selecting the specific columns which we have added above, to check if there are any outliers
    selected_columns = [column for column in df.columns if column.startswith("is_outlier")]

    # Adding all the outlier columns into a new colum "total_outliers", to see the total number of outliers
    df = df.withColumn('total_outliers',sum(df[column] for column in selected_columns))

    # Dropping the extra columns created above, just to create nice dataframe., without extra columns
    df = df.drop(*[column for column in df.columns if column.startswith("is_outlier")])

    return df

In [ ]:
new_data = find_outliers(delete_default_ind_null)
#new_data.show()

new_data_with_no_outliers = new_data.filter(new_data['total_Outliers']<=1)
new_data_with_no_outliers = new_data_with_no_outliers.select(*data.columns)

new_data_with_no_outliers.show()

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+----------+----------+--------------------+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+---------

In [ ]:
new_data_with_no_outliers.count()

855746

**Selecting only several useful features**

In [ ]:
cols = ("desc", "mths_since_last_delinq", "mths_since_last_record", "mths_since_last_major_derog", "annual_inc_joint", "dti_joint", "verification_status_joint", "open_acc_6m", "open_il_6m", "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il", "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc", "all_util", "inq_fi", "total_cu_tl", "inq_last_12m","emp_title","title","zip_code","earliest_cr_line","issue_d","last_pymnt_d","next_pymnt_d","last_credit_pull_d","addr_state")
droppedColumns = new_data_with_no_outliers.drop(*cols)

In [ ]:
len(droppedColumns.columns)

43

In [ ]:
droppedColumns.printSchema()

root
 |-- id: integer (nullable = true)
 |-- member_id: integer (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: string (nullable = true)

In [ ]:
#get the percentage of null values 
amount_missing_df = droppedColumns.select([((count(when(col(c).isNull(), c))/droppedColumns.count())*100).alias(c) for c in droppedColumns.columns])
amount_missing_df.show()

+---+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+----------+--------------+----------+-------------------+----------+-------+---+-----------+--------------+--------+-------+---------+-------------------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+--------------------+--------------------------+--------------------+--------------------+--------------+-----------------+-----------------+------------------+-----------+
| id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|term|int_rate|installment|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|pymnt_plan|purpose|dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|         revol_util|total_acc|initial_list_status|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_f

**Cast column types**

In [ ]:
convertedDf = droppedColumns.withColumn("revol_util", col("revol_util").cast(DoubleType())) \
.withColumn("last_pymnt_amnt", col("last_pymnt_amnt").cast(DoubleType())) \
.withColumn("collections_12_mths_ex_med", col("collections_12_mths_ex_med").cast(IntegerType())) \
.withColumn("policy_code", col("policy_code").cast(IntegerType())) \
.withColumn("tot_coll_amt", col("tot_coll_amt").cast(IntegerType())) \
.withColumn("tot_cur_bal", col("tot_cur_bal").cast(IntegerType())) \
.withColumn("total_rev_hi_lim", col("total_rev_hi_lim").cast(IntegerType()))\


In [ ]:
convertedDf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- member_id: integer (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: string (nullable = true)

In [ ]:
# convertedDfDate = convertedDf.withColumn("last_pymnt_d",to_date(col("last_pymnt_d"),"MM-dd-yyyy").cast(DateType()))\
# .withColumn("next_pymnt_d",to_date(col("next_pymnt_d"),"MM-dd-yyyy").cast(DateType()))\
# .withColumn("last_credit_pull_d",to_date(col("last_credit_pull_d"),"MM-dd-yyyy").cast(DateType()))

# convertedDfDate.printSchema()

In [ ]:
# convertedDfDate.show()

In [ ]:
imputerDouble = ["revol_util","last_pymnt_amnt"]
imputerInteger = ["collections_12_mths_ex_med","policy_code","tot_coll_amt","tot_cur_bal","total_rev_hi_lim"]
# imputerDate = ["last_pymnt_d","next_pymnt_d","last_credit_pull_d"]
# imputerString = ["emp_title","title"]
# "application_type"

In [ ]:
imputer = Imputer(inputCols = imputerDouble, outputCols=["{}_imputed".format(c) for c in imputerDouble])
modelDouble = imputer.fit(convertedDf)
modelDouble.getStrategy()

'mean'

In [ ]:
imputedDoubleCol = modelDouble.transform(convertedDf)
imputedDoubleCol.show()

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+----------+--------------+----------+-------------------+----------+------------------+-----+-----------+--------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+---------------+--------------------------+-----------+----------------+--------------+------------+-----------+----------------+-----------+------------------+-----------------------+
|     id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|      term|int_rate|installment|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|pymnt_plan|           purpose|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|revol_util|total_acc|initial_list_status|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_l

In [ ]:
imputer2 = Imputer(inputCols = imputerInteger, outputCols=["{}_imputed".format(c) for c in imputerInteger])
modelInteger = imputer2.setStrategy("median").fit(imputedDoubleCol)
modelInteger.getStrategy()

'median'

In [ ]:
imputedIntCol = modelInteger.transform(imputedDoubleCol)
imputedIntCol.show()

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+----------+--------------+----------+-------------------+----------+------------------+-----+-----------+--------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+---------------+--------------------------+-----------+----------------+--------------+------------+-----------+----------------+-----------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+
|     id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|      term|int_rate|installment|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|pymnt_plan|           purpose|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|revol_util|to

In [ ]:
#change the null value of the application_type to INDIVIDUAL because that is the mode
fixedApplicationType = imputedIntCol.withColumn("application_type", when (imputedIntCol["application_type"].isNull(), lit("INDIVIDUAL")).otherwise(imputedIntCol["application_type"]) )
fixedApplicationType32 = fixedApplicationType.withColumn("application_type", when (fixedApplicationType["application_type"] == 32, lit("INDIVIDUAL")).otherwise(fixedApplicationType["application_type"]) )

In [ ]:
fixedApplicationType32.groupby('application_type').count().show()

+----------------+------+
|application_type| count|
+----------------+------+
|      INDIVIDUAL|855304|
|           JOINT|   442|
+----------------+------+



In [ ]:
#change the null value to "null"

#christy will fix this tomorrrow, i'm tired
# randomName = fixedApplicationType32.withColumn("emp_title", when (fixedApplicationType32["emp_title"].isNull(), lit("null")).otherwise(fixedApplicationType32["emp_title"]))
# randomTitleName = randomName.withColumn("title", when (randomName["title"].isNull(), lit("null")).otherwise(randomName["title"]))


In [ ]:
fixedApplicationType32.select([count(when(col(c).isNull(), c)).alias(c) for c in fixedApplicationType32.columns]).show()

+---+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+----------+--------------+----------+-------------------+----------+-------+---+-----------+--------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+---------------+--------------------------+-----------+----------------+--------------+------------+-----------+----------------+-----------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+
| id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|term|int_rate|installment|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|pymnt_plan|purpose|dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|revol_util|total_acc|initial_list_status|out_prncp|out_prnc

In [ ]:
#Delete the columns
deleteImputedNumber = ("revol_util","last_pymnt_amnt","collections_12_mths_ex_med","policy_code","tot_coll_amt","tot_cur_bal","total_rev_hi_lim")

imputedDf = fixedApplicationType32.drop(*deleteImputedNumber)

imputedDf.show()


+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+----------+--------------+----------+-------------------+----------+------------------+-----+-----------+--------------+--------+-------+---------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+----------------+--------------+-----------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+
|     id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|      term|int_rate|installment|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|pymnt_plan|           purpose|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|total_acc|initial_list_status|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec

In [ ]:
cleanDf = imputedDf.na.drop()

In [ ]:
cleanDf.select([count(when(col(c).isNull(), c)).alias(c) for c in cleanDf.columns]).show()

+---+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+----------+--------------+----------+-------------------+----------+-------+---+-----------+--------------+--------+-------+---------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+----------------+--------------+-----------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+
| id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|term|int_rate|installment|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|pymnt_plan|purpose|dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|total_acc|initial_list_status|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|a

In [ ]:
cleanDf.count()

855746

In [ ]:
cleanDf.show()

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+----------+--------------+----------+-------------------+----------+------------------+-----+-----------+--------------+--------+-------+---------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+----------------+--------------+-----------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+
|     id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|      term|int_rate|installment|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|pymnt_plan|           purpose|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|total_acc|initial_list_status|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec

**Converting the categorical features**

In [ ]:
categoricalColumns = ("term","grade","sub_grade","emp_length","home_ownership","verification_status","pymnt_plan","initial_list_status","application_type","purpose","acc_now_delinq")

In [ ]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(cleanDf) for column in list(set(cleanDf.columns)-set(['id','member_id','loan_amnt','funded_amnt','funded_amnt_inv','int_rate','installment','emp_title','annual_inc','issue_d','title','zip_code','dti','delinq_2yrs','earliest_cr_line','inq_last_6mths','open_acc','pub_rec','revol_bal','total_acc','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','next_pymnt_d','last_credit_pull_d','cc_now_delinq','default_ind','revol_util_imputed','last_pymnt_amnt_imputed','collections_12_mths_ex_med_imputed','policy_code_imputed','tot_coll_amt_imputed','tot_cur_bal_imputed','total_rev_hi_lim_imputed'])) ]

pipeline = Pipeline(stages=indexers)

df_r = pipeline.fit(cleanDf).transform(cleanDf)

df_r.show()


+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+----------+--------------+----------+-------------------+----------+------------------+-----+-----------+--------------+--------+-------+---------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+----------------+--------------+-----------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+----------------+----------------------+--------------------+-----------+-------------+--------------------+----------------+-------------------------+---------------+----------+-------------------------+
|     id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|      term|int_rate|installment|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|pymnt_p

In [ ]:
#convert the categorical column to integer and then delete the actual columns

categorical_df = df_r.drop(*categoricalColumns)
categorical_df.show()

+-------+---------+---------+-----------+---------------+--------+-----------+----------+-----+-----------+--------------+--------+-------+---------+---------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+-----------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+----------------+----------------------+--------------------+-----------+-------------+--------------------+----------------+-------------------------+---------------+----------+-------------------------+
|     id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|total_acc|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|default_ind|revol_util_impute

In [ ]:
len(categorical_df.columns)

43

In [ ]:
categorical_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- member_id: integer (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- out_prncp: string (nullable = true)
 |-- out_prncp_inv: string (nullable = true)
 |-- total_pymnt: string (nullable = true)
 |-- total_pymnt_inv: string (nullable = true)
 |-- total_rec_prncp: string (nullable = true)
 |-- total_rec_int: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- recoveries: string (nullable = true)
 |-- collection

In [ ]:
#convert it first to int
categorical_df = categorical_df.withColumn("annual_inc", categorical_df["annual_inc"].cast(IntegerType()))\
.withColumn("delinq_2yrs",categorical_df["delinq_2yrs"].cast(IntegerType()))\
.withColumn("inq_last_6mths",categorical_df["inq_last_6mths"].cast(IntegerType()))\
.withColumn("open_acc",categorical_df["open_acc"].cast(IntegerType()))\
.withColumn("pub_rec",categorical_df["pub_rec"].cast(IntegerType()))\
.withColumn("revol_bal",categorical_df["revol_bal"].cast(IntegerType()))\
.withColumn("total_acc",categorical_df["total_acc"].cast(IntegerType()))\
.withColumn("out_prncp",categorical_df["out_prncp"].cast(DoubleType()))\
.withColumn("out_prncp_inv",categorical_df["out_prncp_inv"].cast(DoubleType()))\
.withColumn("total_pymnt",categorical_df["total_pymnt"].cast(DoubleType()))\
.withColumn("total_pymnt_inv",categorical_df["total_pymnt_inv"].cast(DoubleType()))\
.withColumn("total_rec_prncp",categorical_df["total_rec_prncp"].cast(DoubleType()))\
.withColumn("total_rec_int",categorical_df["total_rec_int"].cast(DoubleType()))\
.withColumn("recoveries",categorical_df["recoveries"].cast(DoubleType()))\
.withColumn("total_rec_late_fee",categorical_df["total_rec_late_fee"].cast(DoubleType()))\
.withColumn("collection_recovery_fee",categorical_df["collection_recovery_fee"].cast(DoubleType()))\
.withColumn("dti",categorical_df["dti"].cast(DoubleType()))\



In [ ]:
categorical_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- member_id: integer (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- annual_inc: integer (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: integer (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- out_prncp: double (nullable = true)
 |-- out_prncp_inv: double (nullable = true)
 |-- total_pymnt: double (nullable = true)
 |-- total_pymnt_inv: double (nullable = true)
 |-- total_rec_prncp: double (nullable = true)
 |-- total_rec_int: double (nullable = true)
 |-- total_rec_late_fee: double (nullable = true)
 |-- recoveries: double (nullable = true)
 |-- col

In [ ]:
categorical_df.select([count(when(col(c).isNull(), c)).alias(c) for c in categorical_df.columns]).show()

+---+---------+---------+-----------+---------------+--------+-----------+----------+---+-----------+--------------+--------+-------+---------+---------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+-----------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+----------------+----------------------+--------------------+-----------+-------------+--------------------+----------------+-------------------------+---------------+----------+-------------------------+
| id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|total_acc|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|default_ind|revol_util_imputed|last_pymnt

In [ ]:
finalDataset = categorical_df.na.drop()

In [ ]:
finalDataset.select([count(when(col(c).isNull(), c)).alias(c) for c in finalDataset.columns]).show()

+---+---------+---------+-----------+---------------+--------+-----------+----------+---+-----------+--------------+--------+-------+---------+---------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+-----------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+----------------+----------------------+--------------------+-----------+-------------+--------------------+----------------+-------------------------+---------------+----------+-------------------------+
| id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|total_acc|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|default_ind|revol_util_imputed|last_pymnt

In [ ]:
finalDataset.show()

+-------+---------+---------+-----------+---------------+--------+-----------+----------+-----+-----------+--------------+--------+-------+---------+---------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+-----------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+----------------+----------------------+--------------------+-----------+-------------+--------------------+----------------+-------------------------+---------------+----------+-------------------------+
|     id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|total_acc|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|default_ind|revol_util_impute

In [ ]:
# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=finalDataset.columns, outputCol=vector_col)
df_vector = assembler.transform(finalDataset).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(df_vector, vector_col)

In [ ]:
import pandas as pd

matrix = Correlation.corr(df_vector, "corr_features").collect()[0][0] 
corr_matrix = matrix.toArray().tolist() 
corr_matrix_df = pd.DataFrame(data=corr_matrix) 
corr_matrix_df .style.background_gradient(cmap='coolwarm').set_precision(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
0,1.00,1.00,0.06,0.07,0.07,-0.14,0.02,0.03,0.07,0.05,-0.12,0.09,0.08,0.04,0.03,0.56,0.56,-0.59,-0.59,-0.55,-0.46,-0.08,-0.12,-0.07,-0.22,-0.05,-0.23,0.05,nan,0.01,0.05,0.07,-0.00,0.03,0.02,0.00,-0.06,0.02,-0.02,-0.10,0.00,0.07,0.37
1,1.00,1.00,0.06,0.07,0.07,-0.14,0.02,0.03,0.07,0.05,-0.12,0.09,0.08,0.04,0.03,0.56,0.56,-0.59,-0.59,-0.55,-0.46,-0.08,-0.12,-0.07,-0.22,-0.05,-0.23,0.05,nan,0.01,0.05,0.07,-0.00,0.03,0.02,0.00,-0.06,0.02,-0.02,-0.10,0.00,0.07,0.37
2,0.06,0.06,1.00,1.00,1.00,0.14,0.94,0.34,0.02,-0.00,-0.03,0.20,-0.08,0.34,0.22,0.64,0.64,0.48,0.48,0.40,0.54,0.03,0.08,0.06,-0.00,0.12,0.23,-0.02,nan,-0.00,0.32,0.31,0.00,0.01,0.00,0.15,-0.17,-0.15,-0.07,-0.20,0.14,0.41,0.09
3,0.07,0.07,1.00,1.00,1.00,0.14,0.95,0.34,0.02,-0.00,-0.03,0.20,-0.08,0.34,0.22,0.64,0.64,0.48,0.48,0.39,0.54,0.03,0.08,0.06,-0.01,0.12,0.23,-0.02,nan,-0.00,0.33,0.31,0.00,0.01,0.00,0.15,-0.17,-0.15,-0.07,-0.20,0.14,0.41,0.09
4,0.07,0.07,1.00,1.00,1.00,0.14,0.94,0.33,0.02,0.00,-0.03,0.20,-0.08,0.34,0.22,0.64,0.64,0.47,0.48,0.39,0.53,0.02,0.07,0.05,-0.01,0.12,0.23,-0.02,nan,-0.00,0.33,0.31,0.00,0.01,0.00,0.15,-0.17,-0.15,-0.06,-0.20,0.14,0.41,0.09
5,-0.14,-0.14,0.14,0.14,0.14,1.00,0.13,-0.07,0.08,0.06,0.23,-0.01,0.05,-0.04,-0.04,0.03,0.03,0.17,0.17,0.06,0.45,0.05,0.11,0.07,0.16,0.27,0.11,0.01,nan,0.00,-0.08,-0.16,0.00,0.01,0.03,0.63,0.05,0.05,0.01,-0.12,0.46,0.43,-0.12
6,0.02,0.02,0.94,0.95,0.94,0.13,1.00,0.33,0.01,0.01,0.00,0.18,-0.07,0.32,0.20,0.54,0.54,0.52,0.52,0.46,0.50,0.04,0.08,0.06,0.00,0.13,0.24,-0.01,nan,-0.00,0.29,0.28,0.00,0.01,0.01,0.14,-0.15,-0.13,-0.06,-0.17,0.14,0.14,0.03
7,0.03,0.03,0.34,0.34,0.33,-0.07,0.33,1.00,-0.09,0.05,0.04,0.14,-0.01,0.30,0.19,0.21,0.21,0.16,0.16,0.15,0.13,0.01,0.01,0.01,-0.04,0.04,0.09,-0.00,nan,0.00,0.41,0.26,0.00,-0.01,0.02,-0.01,0.01,-0.13,-0.06,-0.08,0.02,0.06,0.04
8,0.07,0.07,0.02,0.02,0.02,0.08,0.01,-0.09,1.00,-0.00,-0.01,0.14,-0.02,0.07,0.11,0.06,0.06,-0.04,-0.04,-0.05,0.01,-0.01,0.00,0.00,0.00,0.09,-0.03,-0.00,nan,-0.00,-0.00,0.04,0.00,0.08,0.00,0.04,-0.06,0.01,0.00,-0.01,0.02,0.05,0.02
9,0.05,0.05,-0.00,-0.00,0.00,0.06,0.01,0.05,-0.00,1.00,0.02,0.05,-0.01,-0.03,0.12,0.03,0.03,-0.03,-0.03,-0.04,0.00,0.01,-0.00,0.00,-0.01,-0.02,-0.02,0.06,nan,0.00,0.07,-0.04,0.00,0.00,0.13,0.01,0.00,-0.04,-0.03,-0.01,-0.01,-0.00,0.02


In [ ]:
#the 28th column is this
#why null
finalDataset.groupby('policy_code_imputed').count().show()

#the one that we want to predict is the 24th

+-------------------+------+
|policy_code_imputed| count|
+-------------------+------+
|                  1|855743|
+-------------------+------+



In [ ]:
finalDataset.repartition(1).write.csv("/content/drive/MyDrive/CSCI316/CleanWithHeader.csv",header = 'true')

AnalysisException: ignored

In [ ]:
psdf = finalDataset.pandas_api()

In [ ]:
psdf.head(10)

In [ ]:
type(psdf)

In [ ]:
len(psdf)

In [ ]:
psdf.count()

In [ ]:
df_np = []
for i in range(42):
  df_np.append(psdf.iloc[:,i].values)

In [ ]:
fig1, ax1 = plt.subplots()
ax1.boxplot(to_numpyy)

In [ ]:
import numpy as np
to_numpyy = np.array(df_np)

In [ ]:
type(arr_cur_bal)

In [ ]:
arr_cur_bal = psdf['tot_cur_bal_imputed'].values

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Basic Plot')
ax1.boxplot(arr_cur_bal)

In [ ]:
print(len(to_np_arr))

In [ ]:
to_np_arr = psdf.values

In [ ]:
to_np_arr[0]